In [1]:
import numpy as np

In [2]:
def matrix_product(A, B):
    if type(A[0]) != list:
        A = [A]

    if len(A[0]) != len(B):
        raise ValueError("행렬 곱 연산이 불가능합니다.")

    result = [[0 for _ in range(len(B[0]))] for _ in range(len(A))]
    for i in range(len(A)):
        for j in range(len(B[0])):
            dot_product = 0
            for k in range(len(A[0])):
                dot_product += A[i][k] * B[k][j]
            result[i][j] = dot_product
    return result

In [3]:
##########주어짐###########
#입력
X=[[i+0.1,i] for i in range(1000)] #한 시점에 특징이 2가지인 데이터 1000개
#print("X:",np.array(X).shape)
#print(X)
#배치 구현 <- simple_lstm 참고하여 바꿔야
BatchSize = int(input("배치 사이즈:"))
X=X[:BatchSize]
#print("X:",np.array(X).shape)
#print(X)
timeSize = int(input("한번에 고려할 시점 수:")) #한번에 고려할 시점의 개수
newX=[]
for i in range(len(X)-timeSize+1): # [0 1 2 3 4]를 2시점 씩 고려하면 [0 1] [1 2] [2 3] [3 4] 총 4가지가 되므로 
    smallX=[]
    for j in range(timeSize):
        smallX.append(X[i+j])
    newX.append(smallX)
X = newX
print("X:",np.array(X).shape)
print(X)
input_len = len(X)  #데이터의 개수만큼 target과 예측값이 있어야 하므로

#target 한층의 마지막이 아니라 전체 층의 마지막
T=[[i//10] for i in range(input_len)] # 임의로 첫번째 값을 10으로 나눈 몫으로 정함 , 타겟이 마지막 층이라 원소가 1개인 target 값이 input_len만큼있음
print("T:",np.array(T).shape)
print(T)
#!!!!!!!!!!# T가 8,10이여야
#셔플수행하면 좋으나 그냥지나감

X: (93, 8, 2)
[[[0.1, 0], [1.1, 1], [2.1, 2], [3.1, 3], [4.1, 4], [5.1, 5], [6.1, 6], [7.1, 7]], [[1.1, 1], [2.1, 2], [3.1, 3], [4.1, 4], [5.1, 5], [6.1, 6], [7.1, 7], [8.1, 8]], [[2.1, 2], [3.1, 3], [4.1, 4], [5.1, 5], [6.1, 6], [7.1, 7], [8.1, 8], [9.1, 9]], [[3.1, 3], [4.1, 4], [5.1, 5], [6.1, 6], [7.1, 7], [8.1, 8], [9.1, 9], [10.1, 10]], [[4.1, 4], [5.1, 5], [6.1, 6], [7.1, 7], [8.1, 8], [9.1, 9], [10.1, 10], [11.1, 11]], [[5.1, 5], [6.1, 6], [7.1, 7], [8.1, 8], [9.1, 9], [10.1, 10], [11.1, 11], [12.1, 12]], [[6.1, 6], [7.1, 7], [8.1, 8], [9.1, 9], [10.1, 10], [11.1, 11], [12.1, 12], [13.1, 13]], [[7.1, 7], [8.1, 8], [9.1, 9], [10.1, 10], [11.1, 11], [12.1, 12], [13.1, 13], [14.1, 14]], [[8.1, 8], [9.1, 9], [10.1, 10], [11.1, 11], [12.1, 12], [13.1, 13], [14.1, 14], [15.1, 15]], [[9.1, 9], [10.1, 10], [11.1, 11], [12.1, 12], [13.1, 13], [14.1, 14], [15.1, 15], [16.1, 16]], [[10.1, 10], [11.1, 11], [12.1, 12], [13.1, 13], [14.1, 14], [15.1, 15], [16.1, 16], [17.1, 17]], [[11.1, 11]

In [ ]:
#Dense의 parameter 초기화
Dense_out = int(input("Desne의 output의 크기:"))

In [4]:
outputQ = input("LSTM 한 층의 입력하나요(Y/N)?:") #층이 하나면 그냥 target의 종류, 층이 여러개면 정해줘야 함. 현재는 단일 모듈이므로 target의 종류로 정함
count_list=[] # 타겟의 종류 수 카운트 => layer output의 크기를 정하기 위해서(위의 ? 정함) lstm 층이 여러개면 lstm 모듈마다 output_size 정해야
cnt = 0
for now in T:
    if now in count_list:
        pass
    else:
        cnt += 1
        count_list.append(now)
output_size = cnt #w의 output 크기
if(outputQ == "N"):
    output_size = int(input("output_size:")) 
print(output_size)

20


In [5]:
##########작성###########
#state초기화
Y = [[[0 for i in range(output_size)] for i in range(timeSize)] for _ in range(input_len)] # 실제 값은 X의 입력 만큼 있어야 하므로
C = [[[0 for i in range(output_size)] for i in range(timeSize)] for _ in range(input_len)] # X은 시점 수 만큼 있어야 하므로
print("Y:",np.array(Y).shape)
print(Y)
print("C:",np.array(C).shape)
print(C)

Y: (93, 8, 20)
[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [6]:
#parameter초기화
input_feature = len(X[0][0]) # 한 시점에서 데이터의 특징 수 <= X와 w의 행렬곱을 수행해야하므로 X가 (시점수,2)이므로 w는 (2,?)가 되야함
print(input_feature)

2


In [7]:
#parameter 초기화
import random
#가중치 초기화: 가우시안 초기화 방법,  He 초기화 (He Initialization) 또는 Xavier 초기화 (Glorot Initialization)도 존재
w_matrix = []
v_matrix = []
b_matrix = []
for _ in range(4): ## 망각 게이트,입력 게이트,새로운 기억, 출력 게이트 총 4개에서 업데이트 해야함
    w0=[[random.gauss(0,1) for i in range(output_size)] for j in range(input_feature)] # X는 (시점 수, 특징 수)이므로 w 는 (특징수, output_size) 
    #print(np.array(w0).shape)
    v0=[[random.gauss(0,1) for i in range(output_size)]for j in range(output_size)]  # Y는 (시점 수, 특징 수)이므로 w 는 (특징수, output_size)
    #print(np.array(v0).shape)
    b0=[[random.gauss(0,1) for i in range(output_size)]for j in range(timeSize)] 
    #print(np.array(b0).shape)
    w_matrix.append(w0)
    v_matrix.append(v0)
    b_matrix.append(b0)

In [8]:
print(np.array(w_matrix).shape)
print(np.array(v_matrix).shape)
print(np.array(b_matrix).shape)

(4, 2, 20)
(4, 20, 20)
(4, 8, 20)


In [9]:
def element_sum(arr1, arr2):
    if len(arr1) != len(arr2)  or len(arr1[0]) != len(arr2[0]):
        raise ValueError("Input arrays must have the same shape")
    #print(np.array(arr1))
    #print(np.array(arr2))
    result = []
    for i in range(len(arr1)):
        row = []
        for j in range(len(arr1[0])):
            row.append(arr1[i][j] + arr2[i][j])
        result.append(row)
    
    return result

In [10]:
def element_multi(arr1, arr2):
    if len(arr1) != len(arr2) or len(arr1[0]) != len(arr2[0]):
        raise ValueError("Input arrays must have the same shape")
    #print(np.array(arr1))
    #print(np.array(arr2))
    result = []
    for i in range(len(arr1)):
        row = []
        for j in range(len(arr1[0])):
            row.append(arr1[i][j] * arr2[i][j])
        result.append(row)
    return result

In [55]:
def elememt_diff(arr1,arr2):
    if len(arr1) != len(arr2) or len(arr1[0]) != len(arr2[0]):
        raise ValueError("Input arrays must have the same shape")
    #print(np.array(arr1))
    #print(np.array(arr2))
    result = []
    for i in range(len(arr1)):
        row = []
        for j in range(len(arr1[0])):
            row.append(arr1[i][j] - arr2[i][j])
        result.append(row)
    return result

In [34]:
def activation(function,x):
    e = 2.718281828459045 #자연상수:2.71828182845904523536
    result = []
    if(function == 'Sigmoid'):
        for i in range(len(x)):
            row = []
            for j in range(len(x[0])):
                now_x = x[i][j]
                row.append((e**now_x)/((e**now_x)+1)) #C언어에서는 **이 없어 **도 구현
            result.append(row)
        return result
    if(function == 'Tanh'):
        for i in range(len(x)):
            row = []
            for j in range(len(x[0])):
                now_x = x[i][j]
                row.append((2/(1+(e**(-2*now_x))))-1)
            result.append(row)
        return result
    if(function == 'ReLU'):
        for i in range(len(x)):
            row = []
            for j in range(len(x[0])):
                now_x = x[i][j]
                if (now_x<=0):
                    row.append(0)
                else:
                    row.append(now_x)
            result.append(row)
        return result
    if(function == 'Equl'):
        return x

In [17]:
def LSTMforward(nowtime,X,w_matrix,Y,v_matrix):
    ###################################LSTM###################################
    pasttime = nowtime - 1
    if(nowtime == 0):
        pasttime += 1
    #forgot gate
    #print("===========forgot gate============")
    Xw = matrix_product(X[nowtime],w_matrix[0])
    Yv = matrix_product(Y[pasttime],v_matrix[0])
    Xw_Yv_b = element_sum(element_sum(Xw,Yv),b_matrix[0])
    fGate = activation('Sigmoid',Xw_Yv_b)
    #print(np.array(fGate).shape)
    #print(np.array(fGate))

    #input gate
    #print("===========input gate============")
    Xw = matrix_product(X[nowtime],w_matrix[1])
    Yv = matrix_product(Y[pasttime],v_matrix[1])
    Xw_Yv_b = element_sum(element_sum(Xw,Yv),b_matrix[1])
    iGate = activation('Sigmoid',Xw_Yv_b)
    #print(np.array(iGate).shape)
    #print(np.array(iGate))

    #new input
    #print("===========new input============")
    Xw = matrix_product(X[nowtime],w_matrix[2])
    Yv = matrix_product(Y[pasttime],v_matrix[2])
    Xw_Yv_b = element_sum(element_sum(Xw,Yv),b_matrix[2])
    newIn = activation('Tanh',Xw_Yv_b)
    #print(np.array(newIn).shape)
    #print(np.array(newIn))

    #output gate
    #print("===========output gate============")
    Xw = matrix_product(X[nowtime],w_matrix[3])
    Yv = matrix_product(Y[pasttime],v_matrix[3])
    Xw_Yv_b = element_sum(element_sum(Xw,Yv),b_matrix[3])
    oGate = activation('Sigmoid',Xw_Yv_b)
    #print(np.array(oGate).shape)
    #print(np.array(oGate))

    #State
    C[nowtime] = element_sum(element_multi(C[pasttime], fGate),element_multi(iGate, newIn))
    Y[nowtime] = element_multi(oGate,activation('Tanh',C[nowtime]))

    #print(nowtime)
    #print(np.array(Y[nowtime]))


In [18]:
#순전파 LSTM
for time in range(input_len): #batchsize 만큼 forward 진행해야 
    LSTMforward(time,X,w_matrix,Y,v_matrix)

In [19]:
print(np.array(Y))
print(np.array(Y).shape)

[[[-4.97810541e-02 -1.09123118e-01  2.52515345e-01 ... -4.05911403e-02
    4.36453678e-02  2.63948259e-01]
  [ 2.25763818e-02 -1.56726834e-01 -1.50520670e-01 ... -5.04848632e-01
   -2.84718753e-01  3.45190003e-01]
  [ 8.12219931e-03 -9.80878143e-02 -5.51839047e-01 ... -1.36836363e-01
    1.84340582e-01  3.54302341e-01]
  ...
  [ 2.61736403e-05 -1.77961115e-02 -8.12775977e-01 ... -2.01186398e-01
   -1.57203771e-02  4.10369346e-02]
  [ 5.21283838e-05 -7.40652975e-03 -8.09363208e-01 ... -6.63149928e-01
   -3.23066225e-02  9.71580159e-04]
  [ 1.30493154e-05  4.40057702e-03 -9.23796466e-01 ... -5.00668967e-01
   -2.68819233e-02  7.06922860e-03]]

 [[ 2.84728026e-03 -3.18284999e-01 -3.19923279e-01 ... -1.11111109e-01
    5.80447513e-02  3.03885964e-01]
  [ 1.02006492e-02 -8.55165343e-02 -1.88921387e-01 ... -6.74545975e-01
   -3.24278519e-01  1.72231648e-01]
  [ 5.56817532e-03  2.26219402e-01 -4.08435433e-01 ... -6.11803343e-01
    5.84841329e-02  3.22294640e-01]
  ...
  [ 4.60013149e-06 -1.9

In [20]:
print(np.array(Y[input_len-1]))
print(np.array(Y[input_len-1]).shape)

[[ 1.36921774e-62  9.22589318e-30 -1.00000000e+00 -7.04295263e-11
  -7.61594156e-01  0.00000000e+00  3.71158219e-63 -1.76450180e-59
   0.00000000e+00  0.00000000e+00 -9.13695975e-25  9.49828430e-01
  -5.85355890e-13  1.28700819e-69  4.49388595e-01  5.85295675e-85
  -3.63457915e-23 -1.63254401e-07  0.00000000e+00  6.08052930e-38]
 [ 7.03481160e-54  6.16408382e-31 -1.00000000e+00 -1.48214802e-12
  -7.61594156e-01  0.00000000e+00  6.46230618e-64 -3.33953832e-60
   0.00000000e+00  0.00000000e+00 -2.28099533e-25  1.25902334e-01
  -3.59959698e-12  2.08943975e-68  3.72521907e-01 -2.00939354e-84
   4.23261802e-23 -3.40143013e-06  0.00000000e+00  6.01719517e-39]
 [ 6.23369995e-52  2.60325596e-30 -1.00000000e+00 -1.17127310e-11
  -7.61594156e-01  0.00000000e+00  1.06653594e-63 -3.32192484e-60
   0.00000000e+00  1.30932449e-24 -8.10924755e-26  6.44930949e-01
  -1.57858460e-13  4.73981984e-70  9.59464879e-01 -1.22196013e-85
   2.63242517e-24 -9.16873883e-07  0.00000000e+00  6.74358351e-39]
 [ 0.00

In [35]:
#순전파 Dense
Dense_X = Y[input_len-1] #LSTM수행 후 마지막 Y값을 Dense의 입력값(X)으로
print(np.array(Dense_X).shape)

(8, 20)


In [36]:
#Dense의 parameter 초기화
Dense_out = int(input("Desne의 output의 크기:"))

In [37]:
Dense_X_feature = 0
Dense_X_time = 0
for i in Dense_X:
    Dense_X_time +=1
for i in Dense_X[0]:
    Dense_X_feature += 1

In [38]:
import random
#가중치 초기화: 가우시안 초기화 방법,  He 초기화 (He Initialization) 또는 Xavier 초기화 (Glorot Initialization)도 존재
Dense_w = [[random.gauss(0,1) for i in range(Dense_out)] for j in range(Dense_X_feature)]
Dense_b = [[0 for i in range(Dense_out)] for j in range(Dense_X_time)]

print(np.array(Dense_w).shape)
print(np.array(Dense_b).shape)

(20, 10)
(8, 10)


In [51]:
Dense_Y = element_sum(matrix_product(Dense_X,Dense_w),Dense_b)

In [52]:
print(np.array(Dense_Y).shape)
print(np.array(Dense_Y))

(8, 10)
[[ 2.11408146e-01 -1.00797232e+00 -1.11818922e-03 -2.67380158e+00
   3.57212984e-01  7.50657796e-01 -1.15405717e+00 -4.78304818e-01
   6.82706046e-01  4.05316242e-01]
 [-3.94957964e-01  1.08680765e-01  9.15215896e-01 -1.58220961e+00
   5.22181113e-01  1.56591487e+00 -5.35997407e-01 -8.64719972e-01
   4.90495667e-01  1.54241534e+00]
 [ 7.39029932e-01 -4.19758853e-01  7.40985875e-01 -2.04604228e+00
   1.14777590e+00  1.11300676e+00 -2.08032922e-01 -8.63214133e-01
   7.91336585e-01  1.43386440e+00]
 [ 1.70981686e-01 -2.83648646e-01  7.01347338e-01 -1.94375569e+00
   7.57388636e-01  1.25152344e+00 -4.94653945e-01 -8.04515479e-01
   6.45469990e-01  1.32187059e+00]
 [ 2.35842822e-01  1.58863606e-01  1.15793268e+00 -1.49217181e+00
   1.07626670e+00  1.55042362e+00 -2.62009251e-02 -1.02298286e+00
   6.44912008e-01  1.92724126e+00]
 [ 5.32540291e-02 -2.57192849e-01  6.91402259e-01 -1.92432974e+00
   6.75431369e-01  1.27902653e+00 -5.55723497e-01 -7.91539801e-01
   6.15305103e-01  1.2963

In [53]:
#Denseforward의 활성화 함수
Dense_Y = activation("Equl",Dense_Y)

In [54]:
print(np.array(Dense_Y).shape)
print(np.array(Dense_Y))

(8, 10)
[[ 2.11408146e-01 -1.00797232e+00 -1.11818922e-03 -2.67380158e+00
   3.57212984e-01  7.50657796e-01 -1.15405717e+00 -4.78304818e-01
   6.82706046e-01  4.05316242e-01]
 [-3.94957964e-01  1.08680765e-01  9.15215896e-01 -1.58220961e+00
   5.22181113e-01  1.56591487e+00 -5.35997407e-01 -8.64719972e-01
   4.90495667e-01  1.54241534e+00]
 [ 7.39029932e-01 -4.19758853e-01  7.40985875e-01 -2.04604228e+00
   1.14777590e+00  1.11300676e+00 -2.08032922e-01 -8.63214133e-01
   7.91336585e-01  1.43386440e+00]
 [ 1.70981686e-01 -2.83648646e-01  7.01347338e-01 -1.94375569e+00
   7.57388636e-01  1.25152344e+00 -4.94653945e-01 -8.04515479e-01
   6.45469990e-01  1.32187059e+00]
 [ 2.35842822e-01  1.58863606e-01  1.15793268e+00 -1.49217181e+00
   1.07626670e+00  1.55042362e+00 -2.62009251e-02 -1.02298286e+00
   6.44912008e-01  1.92724126e+00]
 [ 5.32540291e-02 -2.57192849e-01  6.91402259e-01 -1.92432974e+00
   6.75431369e-01  1.27902653e+00 -5.55723497e-01 -7.91539801e-01
   6.15305103e-01  1.2963

In [ ]:
### Dense 역전파
Dense_delta = Dense_Y - T


In [ ]:
# LSTM역전파를 위한 parameter의 기울기(미분값) 초기화 #reset_sum_grad
grad_w = []
grad_v = []
grad_b = []
for _ in range(4): ## 망각 게이트,입력 게이트,새로운 기억, 출력 게이트 총 4개에서 업데이트 해야함
    w0=[[0 for i in range(output_size)] for j in range(input_feature)] # X는 (시점 수, 특징 수)이므로 w 는 (특징수, output_size) 
    #print(np.array(w0).shape)
    v0=[[0 for i in range(output_size)]for j in range(output_size)]  # Y는 (시점 수, 특징 수)이므로 w 는 (특징수, output_size)
    #print(np.array(v0).shape)
    b0=[[0 for i in range(output_size)]for j in range(timeSize)] 
    #print(np.array(b0).shape)
    grad_w.append(w0)
    grad_v.append(v0)
    grad_b.append(b0)


In [ ]:
print(np.array(grad_w).shape)
print(np.array(grad_v).shape)
print(np.array(grad_b).shape)

In [ ]:
for time in reversed(range(input_len)):
    LSTMbackward(time,X,w_matrix,Y,v_matrix)

In [ ]:
def LSTMbackward(nowtime):
    than_c = activation('Tanh',C[nowtime])